# Strands Agents with AgentCore Memory (Long-term) using tools

日本語訳:

# ツールを使用した AgentCore メモリ (長期) を持つ Strands エージェント

Strands エージェントは、長期的な記憶を持つ AgentCore を使用しています。これにより、エージェントは以前の対話を覚えており、コンテキストに基づいて応答できます。また、エージェントはツールを使用して、さまざまなタスクを実行できます。たとえば、 search_quality_reflection ツールを使用して、検索結果の質を評価したり、 search_quality_score を使用して検索結果の質を 0 ～ 5 の範囲で数値化したりできます。 result_quality_reflection ツールを使用すると、最終的な結果の質を評価できます。

## 概要

このノートブックでは、Strands と AgentCore Memory を使用して会話 AI エージェントに長期記憶機能を実装する方法を示します。短期的な対話から重要な情報を抽出して統合する方法を学び、エージェントが複数の会話セッションにわたって重要な詳細を想起できるようになります。

## チュートリアルの詳細

**ユースケース:** 永続的なメモリを持つ料理アシスタント

| 情報                | 詳細                                                                              |
|:--------------------|:----------------------------------------------------------------------------------|
| チュートリアルの種類 | 長期会話                                                                         |
| エージェントの種類   | 料理アシスタント                                                                 |
| エージェントフレームワーク | Strands Agents                                                                   |
| LLM モデル          | Anthropic Claude Sonnet 3.7                                                      |
| チュートリアルコンポーネント | AgentCore 'User Preferences' Memory Extraction、メモリの保存と取得のための Memory Tool |
| 例の複雑さ          | 初級                                                                             |

以下のことを学びます:
- 長期保持のための抽出戦略を使用して AgentCore Memory を構成する
- 前の会話履歴でメモリを初期化する
- 長期記憶を使用して、会話セッションにわたって個人化された体験を提供する
- Strands Agent Framework と AgentCore Memory ツールを統合する

## シナリオのコンテキスト

このチュートリアルでは、非常に個人化されたレストラン推薦を提供するように設計された料理アシスタントの役割を担います。AgentCore Memory の長期保持と自動情報抽出機能を活用することで、エージェントはユーザーの食事制限や好みの料理などの嗜好を複数の会話にわたって記憶できます。この永続的なメモリにより、会話が数日または数週間に及んでも、エージェントはユーザーに合わせた提案とシームレスな体験を提供できます。このシナリオでは、構造化されたメモリ構成と設定可能な戦略が、会話 AI に短期的な想起を超えた真に魅力的で状況を認識したインタラクションを実現する力を与えることを示しています。

## アーキテクチャ

<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>

## 前提条件

このチュートリアルを実行するには、以下が必要です:
- Python 3.10 以上
- Amazon Bedrock AgentCore Memory の許可を持つ AWS 認証情報
- Amazon Bedrock AgentCore SDK

適切な抽出戦略を使用して長期メモリリソースを作成し、環境をセットアップしましょう!

## ステップ 1: 環境設定
必要なすべてのライブラリをインポートし、このノートブックを動作させるためのクライアントを定義しましょう。

In [ ]:
!pip install -qr requirements.txt

In [ ]:
import time
import logging
import time
from datetime import datetime

以下が日本語訳になります。

Amazon Bedrock models と AgentCore に適切な権限を持つリージョンとロールを定義してください。

In [ ]:
import os

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S")
logger = logging.getLogger("culinary-memory")

region = os.getenv('AWS_REGION', 'us-west-2')

## ステップ 2: 長期戦略を使ってメモリを作成する

このセクションでは、長期メモリ機能を備えたメモリリソースを作成します。前の短期メモリの例とは異なり、この実装には統合された情報を保持できる特定のメモリ戦略が含まれています。

In [ ]:
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType

client = MemoryClient(region_name=region)

memory_name = "CulinaryAssistant"
memory_id = None

In [ ]:
from botocore.exceptions import ClientError

try:
    print("Creating Long-Term Memory...")

    # We use a more descriptive name for our long-term memory resource

長期記憶リソースにより説明的な名前を使用します。
    memory_name = memory_name

    # Create memory with user preference strategy
    memory = client.create_memory_and_wait(
        name=memory_name,
        description="Culinary Assistant Agent with long term memory",
        strategies=[{
                    StrategyType.USER_PREFERENCE.value: {
                        "name": "UserPreferences",
                        "description": "Captures user preferences",
                        "namespaces": ["user/{actorId}/preferences"]
                    }
                }],
        event_expiry_days=7,
        max_wait=300,
        poll_interval=10
    )

    memory_id = memory['memoryId']
    print(f"Memory created successfully with ID: {memory_id}")
    
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # If memory already exists, retrieve its ID

メモリがすでに存在する場合は、その ID を取得します。
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
except Exception as e:
    # メモリ作成中のエラーを処理する

try :
    memory = create_memory ( )
except MemoryError as error :
    print ( f"Error creating memory: { error }" )
    sys.exit ( 1 )

try :
    data = load_data ( )
    memory.load ( data )
except ( DataError , MemoryError ) as error :
    print ( f"Error loading data into memory: { error }" )
    sys.exit ( 1 )

print ( "Memory loaded successfully." )
    logger.info(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()
    # エラー時の後処理 - 部分的に作成された場合はメモリを削除する

日本語訳:
エラー発生時の後処理として、メモリが部分的に作成された場合は削除します。 malloc() 関数が失敗した場合、つまり NULL ポインタが返された場合は、すでに割り当てられたメモリを解放する必要があります。そのため、以下のように処理を行います。

```c
void * ptr = malloc(size);
if (ptr == NULL) {
    /* エラー処理 */
    return NULL;
}

/* 部分的に割り当てられたメモリを解放する */
for (int i = 0; i < num_chunks; i++) {
    if (chunks[i] != NULL) {
        free(chunks[i]);
    }
}

free(ptr);
return NULL;
```

この例では、 malloc() が失敗した場合、すでに割り当てられた chunks[i] のメモリを解放し、最後に ptr のメモリを解放しています。このようにして、リソースの漏れを防ぐことができます。
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.info(f"Failed to clean up memory: {cleanup_error}")

### 長期メモリ戦略の理解

この記憶の作成における重要な違いは、**メモリ戦略**の追加です。構成要素を分解しましょう。

#### 1. ユーザー設定メモリ戦略

この戦略は、会話からユーザーの設定を自動的に特定して抽出します。

```python
"userPreferenceMemoryStrategy": {
    "name": "UserPreferences",
    "description": "Captures user preferences",
    "namespaces": ["user/{actorId}/preferences"]
}
```

#### 2. メモリ名前空間

`namespaces` パラメータは、抽出された情報が格納される場所を定義します。

```python
"namespaces": ["user/{actorId}/preferences"]
```

この memory 戦略は、単に会話を記憶するだけでなく、その会話の中の重要な情報を実際に理解し、将来の使用のために整理する、より洗練されたメモリシステムを作り出します。

## ステップ 3: 過去の会話をメモリに保存する

このセクションでは、短期メモリを水和する方法を示します。これにより、バックグラウンドで長期メモリの抽出プロセスが自動的にトリガーされます。

### 短期メモリの水和

会話を抽出戦略で構成されたメモリリソースに保存すると、システムは追加のコードを必要とせずに、この情報を自動的に処理して長期保持します。

In [ ]:
actor_id = f"user-{datetime.now().strftime('%Y%m%d%H%M%S')}"
session_id = f"foodie-{datetime.now().strftime('%Y%m%d%H%M%S')}"
namespace = f"user/{actor_id}/preferences"

In [ ]:
previous_messages = [
    ("Hi, I'm John", "USER"),
    ("Hi John, how can I help you with food recommendations today?", "ASSISTANT"),
    ("I'm looking for some vegetarian dishes to try this weekend.", "USER"),
    ("That sounds great! I'd be happy to help with vegetarian recommendations. Do you have any specific ingredients or cuisine types you prefer?", "ASSISTANT"),
    ("Yes, I really like tofu and fresh vegetables in my dishes", "USER"),
    ("Perfect! Tofu and fresh vegetables make for excellent vegetarian meals. I can suggest some stir-fries, Buddha bowls, or tofu curries. Do you have any other preferences?", "ASSISTANT"),
    ("I also really enjoy Italian cuisine. I love pasta dishes and would like them to be vegetarian-friendly.", "USER"),
    ("Excellent! Italian cuisine has wonderful vegetarian options. I can recommend pasta primavera, mushroom risotto, eggplant parmesan, or penne arrabbiata. The combination of Italian flavors with vegetarian ingredients creates delicious meals!", "ASSISTANT"),
    ("I spent 2 hours looking through cookbooks but couldn't find inspiring vegetarian Italian recipes", "USER"),
    ("I'm sorry you had trouble finding inspiring recipes! Let me help you with some creative vegetarian Italian dishes. How about stuffed bell peppers with Italian herbs and rice, spinach and ricotta cannelloni, or a Mediterranean vegetable lasagna?", "ASSISTANT"),
    ("Hey, I appreciate food assistants with good taste", "USER"),
    ("Ha! I definitely try to bring good taste to the table! Speaking of which, shall we explore some more vegetarian Italian recipes that might inspire you?", "ASSISTANT")
]

In [ ]:
print("\nHydrating short term memory with previous conversations...")

# 会話履歴をショートターム メモリに保存する

日本語訳:

# 会話履歴を short-term memory に保存します
initial = client.create_event(
    memory_id=memory_id,
    actor_id=actor_id,
    session_id=session_id,
    messages=previous_messages,
)
print("✓ Conversation saved in short term memory")

会話メッセージを含むイベントが正しく保存されたことを確認しましょう。

In [ ]:
events = client.list_events(
    memory_id=memory_id,
    actor_id=actor_id,
    session_id=session_id,
    max_results=5
)
events

以下が日本語訳になります。

この セル は、実行中に情報を伝える メッセージ を 表示する よう ロギング システム を 設定し ます。これにより、コード が 実行される 際に 何が 起こっている か を 追跡する の に 役立ち ます。

### 裏側で起こること

`create_event` の呼び出し後、以下が自動的に発生します。

1. **短期ストレージ**: 完全な会話が生のままで保存されます
2. **抽出トリガー**: メモリシステムは、このメモリに UserPreference 戦略が設定されていることを検出します
3. **バックグラウンド処理**: 追加のコードなしで、システムは以下を行います。
   - 会話を分析して嗜好指標を探します
   - "私は菜食主義者です" や "本当にイタリア料理が好きです" などの発言を特定します
   - これらの嗜好を構造化されたデータとして抽出します
4. **長期統合**: 抽出された嗜好は、設定された名前空間 (`user/{actorId}/preferences`) に保存されます

抽出と統合は自動的に行われます。エージェントとの会話を維持するか、短期メモリを注入するだけで、メモリ作成時に設定した戦略が残りの処理を行います。

この自動プロセスにより、短期会話記録の有効期限が切れた後でも、重要な情報が長期メモリに保存されます。

## 長期記憶の取得

このセクションでは、長期記憶に保存された抽出された設定を取得する方法を探ります。会話のターンに焦点を当てる短期記憶の取得とは異なり、長期記憶の取得は抽出され統合された構造化された情報にアクセスすることに焦点を当てています。

### 長期メモリからユーザー設定にアクセスする

長期メモリから情報を取得するには、メモリ作成時に定義された namespace 構造を使用します。

日本語訳:

In [ ]:
# Adding a 30s wait to ensure the memory extraction has time to process the event

日本語訳:
# メモリ抽出がイベントを処理するのに十分な時間があることを確認するために 30 秒の待ち時間を追加します
time.sleep(30)

try:
    # Query the memory system for food preferences

メモリシステムに食べ物の好みを問い合わせる
    food_preferences = client.retrieve_memories(
        memory_id=memory_id,
        namespace=namespace,
        query="food preferences",
        top_k=3  # Return up to 3 most relevant results

この行は翻訳せずにそのまま残します。

上記の行は、コードやコマンドの一部であると判断されるためです。 

コードやコマンド、変数名、関数名などの技術的な用語は翻訳せずにそのまま残すよう、指示されています。
    )

    if food_preferences:
        print(f"Retrieved {len(food_preferences)} relevant preference records:")
        for i, record in enumerate(food_preferences):
            print(f"\nMemory {i+1}:")
            print(f"- Content: {record.get('content', 'Not specified')}")
    else:
        print("No matching preference records found.")

except Exception as e:
    print(f"Error retrieving preference records: {e}")

以下が日本語訳になります。

この方法により、必要に応じて関連する記憶を取り出すことができます。基本を学んだので、次は私たちのエージェントを構築しましょう!

## ステップ 4: エージェントの作成
このセクションでは、ネイティブの `agent_core_memory` ツールを使用して、AgentCore Memory を Strands エージェントと統合する方法を説明します。

#### 長期記憶機能を備えたエージェントのセットアップ
記憶機能付きのエージェントを作成するには、Strands フレームワークを使用し、AgentCore Memory リソースに接続します。

In [ ]:
from strands import tool, Agent
from strands_tools.agent_core_memory import AgentCoreMemoryToolProvider

In [ ]:
system_prompt = f"""あなたは Culinary Assistant という高度な飲食店推薦アシスタントです。

目的:
- ユーザーの好みに基づいて飲食店を見つけるのを支援する
- 会話を通してユーザーの好みを記憶する
- パーソナライズされた飲食店の推薦を行う

"""

In [ ]:
provider = AgentCoreMemoryToolProvider(
    memory_id=memory_id,
    actor_id=actor_id,
    session_id=session_id,
    namespace=namespace
)

agent = Agent(tools=provider.tools, system_prompt=system_prompt)

私たちはすでに短期記憶と長期記憶を構築しましたので、エージェントから直接記憶を取り出しましょう!

日本語訳: 短期記憶と長期記憶を既に構築済みですので、エージェントから直接 memory を取り出しましょう!

In [ ]:
agent("Give me restaurant recommendations in Irvine based on my food preferences")

エージェントは retrieve_memory_records メソッドを使用して、ユーザーの記憶を取得する必要がありました。

素晴らしい! あなたは AgentCore の長期記憶から記憶を取り出すことができる Strands Agent を手に入れました!

## リソースの解放
このノートブックで使用したリソースを解放するため、メモリを削除しましょう。

In [ ]:
#client.delete_memory_and_wait(

日本語訳:
# client.delete_memory_and_wait( は、技術的な用語のため翻訳せずにそのまま残します。この関数は、おそらくクライアントのメモリを削除し、その処理が完了するまで待機することを意味しています。
#        memory_id = memory_id,

日本語訳:

#        memory_id = memory_id,
#        max_wait = 300,

日本語訳:
#        max_wait = 300,
#        poll_interval =10

日本語訳:

#        poll_interval = 10
翻訳するテキスト:

# )

日本語訳:

# ) は空のコメントです。